In [1]:
%run input/Format.ipynb
!rm output/dynamic_delta_fits/?/*
!rm output/dynamic_delta_fits/??/*
import ROOT as root
from array import array
import math as math
root.gErrorIgnoreLevel = root.kFatal
import multiprocessing as mp
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0
is_sigmalised = 1
sim = 0
new = 1
Min_Entries = 10
N_files = 1

In [ ]:
isigma = 2
N_layers = 16
N_hists = 4
N_phi = 6
N_ilyaers = 1
is_e = 1
layers = ["1-2 layer east pos","1-2 layer west pos","1-2 layer east neg","1-2 layer west neg","2-3 layer east pos","2-3 layer west pos",\
          "2-3 layer east neg","2-3 layer west neg","2-4 layer east pos","2-4 layer west pos","2-4 layer east neg","2-4 layer west neg",\
          "3-4 layer east pos","3-4 layer west pos","3-4 layer east neg","3-4 layer west neg",\
          "3 layer east pos","3 layer west pos","3 layer east neg","3 layer west neg",\
          "4 layer east pos","4 layer west pos","4 layer east neg","4 layer west neg",\
          "20-40%","40-60%","60-80%","80-93%"]
phi_bins = [35,35,40,40,47,47,52,52,59,59,65,65]
phi_bins = [91,91,96,96,100,100,101,101,105,105,110,110]
if new: phi_bins = [45,45,47,47,49,49,52,52,54,54,56,56]
if is_sigmalised: phi_bins = [45,45,46,46,48,50,51,53,55,55,56,56]
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
file_path="input/NewHadron/"
file_names="my-m_ee_Run14AuAu_74th_new_19784_984runs" 
#m_ee_Run14AuAu_74th_new_19784_984runs m_ee_Run14AuAu_76th_new_19812_75runs
#m_ee_Run14AuAu_29th_new_19536_1078runs m_ee_Run14AuAu_73rd_new_19777_990runs
#m_ee_Run14AuAu_62nd_new_19697_798runs
#m_ee_Run14AuAu_52nd_new_19642_672runs m_ee_Run14AuAu_61st_new_19692_328runs
# m_ee_Run14AuAu_29th_new_19536_90runs 17th_QA_19443_all_rg 18th_QA_19445_all_rg my-17th_QA_19443_all_rg my-25th_QA_19197_1000runs
if not new:
    file_path="input/NewHadron/"
    file_names="missalign_hadron_new8_rg0-8"
hist_select_3D_names = ["dphi_hist","dthe_hist","sdphi_hist","sdthe_hist"]
if is_e == 1: hist_select_3D_names = ["dphi_hist_el_dynamic","dthe_hist_el_dynamic","sdphi_hist_el_dynamic","sdthe_hist_el_dynamic"]
iOptames=["phi","theta"]

In [4]:
if sim :
    file_path="../sim/output/Newembed/deltas/" 
    file_path="../sim/output/Newembed/sngl/" 
    file_path="../sim/output/Newembed/deltas/250301/" 
    file_names="dalitz_50M_v3"#"bbbar_10M_v0" #piminus_pythia_1M_v0 piminus_DCA_50M_V3 photon_100M_V0 piminus_DCA_50M_V1 piminus_DCA_50M_V0_100mrad ccbar_50M_v0 piminus_DCA_50M_V0
    file_names="lepton_50M_v1"

In [5]:
hists_read = []

for iFile in range(-1,N_files):
    infile = root.TFile.Open(file_path+file_names+f".root", "read")
    hists_read0 = []
    for ilayer in range(N_layers):
        hist_select_3D0 = []
        for ihist in range(len(hist_select_3D_names)):
            hist_select_3D0.append(infile.Get(hist_select_3D_names[ihist]+f"_{ilayer}"))
            hist_select_3D0[-1].SetDirectory(root.nullptr)
        hists_read0.append(hist_select_3D0)
    hists_read.append(hists_read0)

infile.Close()

In [6]:
hist_for_pt = hists_read[0][0][iOption0]
for ilayer in range(1,N_layers):
    hist_for_pt.Add(hists_read[0][ilayer][iOption0])
pt_proj = hist_for_pt.ProjectionZ()
c0 = root.TCanvas(f"c0",f"c0",720,360)
Format_Hist_total(pt_proj,"p_{T}","dN/dp_{T}",left=0.15, bottom=0.15, right=0, top=0,  Tsize=0.07,  Lsize=0.06,\
                      Mstyle=21,  Msize=1, Mcolor=4,  Lwidth=3,  Lcolor=4,  offset_x=1, offset_y=1, title="",  Malpha=1,  Lalpha=1)
pt_proj.GetXaxis().SetRange(3,49)
c0.SetLogy()
pt_proj.Draw("P")
myexp = root.TF1("myexp","([0]*0.1*([1]-1)*([1]-2))*(pow((sqrt(x*x+[3]*[3])+[2]-[3])/([2]),-[1]))/(2*3.14159)/([2]+[3]*([1]-2))/([2]+[3])",0.2,5)
myexp.SetParameter(3,0.135)
myexp.SetParameter(0,pt_proj.GetMaximum()*12)
myexp.SetParameter(1,15)
myexp.SetParameter(2,4.7)
pt_proj.Fit(myexp,"Q","",0.4,5)
c0.Draw()

In [7]:
pt = array( 'd' )
pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.3,1.7,2.2,3.0,4.9])
if is_sigmalised: pt_boarders = array( 'd', [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.3,1.7,2.2,3.0,4.9])
pt_err = array( 'd' )
N_pt = len(pt_boarders) - 1
N_steps = 100
for i in range(N_pt):
    pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(0+0.5)
    for istep in range(N_steps):
        pt_step = pt_boarders[i]+(pt_boarders[i+1]-pt_boarders[i])/N_steps*(istep+0.5)
        if myexp.Integral(pt_boarders[i],pt_step,1e-5) > myexp.Integral(pt_step,pt_boarders[i+1],1e-5):
            break
    pt.append( round(pt_step,4))
    pt_err.append((pt_boarders[i+1]-pt_boarders[i])/2)
print("pt = ", pt,"\npt_boarders = ",pt_boarders)
if not is_sigmalised:  pt[0]=0.225

pt =  array('d', [0.2395, 0.3415, 0.4415, 0.5425, 0.6435, 0.7435, 0.8445, 0.9445, 1.1065, 1.438, 1.8675, 2.444, 3.3895]) 
pt_boarders =  array('d', [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.3, 1.7, 2.2, 3.0, 4.9])


In [8]:
boards_tot = [[-0.099,0.099],[-0.099,0.099]]
boards_fit = [[-0.011,0.012],[-0.011,0.012]]
boards_fit2 = [[-0.03,-0.01],[0.015,0.035],[-0.09,-0.03],[0.035,0.09],[-0.09,-0.03],[0.035,0.09],[-0.06,-0.025],[0.03,0.065]]
boards_fit3 = [[-0.02,0.02],[-0.02,0.02]]
if is_sigmalised==1:
    boards_tot = [[-5,5],[-5,5]]
    boards_fit = [[-1,1],[-1,1]]
if is_sigmalised == 1:
    boards_tot = [[-10,10],[-10,10]]
    boards_fit = [[-1,1],[-1,1]]
    boards_fit2 = [[-6,-3],[3,6],[-6,-2],[2,6],[-6,-2],[2,6],[-6,-2],[2,6]]
    boards_fit3 = [[3,7],[-7,-3]]
arms = N_layers/8
leftright = [[1,10],[1,10]]
leftright_sigma = [[1,100],[1,100]]

In [9]:
def sdphi_fitter(InHist, iphi, iFile=0, ilayer=0, sigma=2, fits_mean=0, fits_sigma=0, accur = 0.5):
    
    N_gaus=2
    seed0=iFile*N_pt*ilayer*100
    iilayer = int(ilayer/arms)
    charge = (iilayer%2)*2 - 1
    factor1, factor2 = 1 + int(iilayer>0), 1 + 5*int(iilayer>0)
    hist_pt_bin = InHist.ProjectionZ("kek")

    pt_local =  array( 'd' )
    pt_local_err =  array( 'd' )

    central_bean_prev = (0.001*phi_bins[2*iphi]-0.0005-0.1)
    if new: central_bean_prev = (0.002*phi_bins[2*iphi]-0.001-0.1)
    if is_sigmalised : central_bean_prev = 0
    Mean_local =  array( 'd' )
    Sigma_local =  array( 'd' )
    Mean_local_err =  array( 'd' )
    Sigma_local_err =  array( 'd' )

    int_fg_local =  array( 'd' )
    int_bg_local =  array( 'd' )
 
    for p in range(N_pt):
        c1 = root.TCanvas(f"c1{seed0+p}",f"c1{seed0+p}",720,360)

        m2_name=str(ilayer)+"/"+"F"+str(iFile)+"_P"+str(iphi)+"_dynamic_"+str(round(pt[p],1))

        var_porj = InHist.ProjectionX(f"sdphi{seed0+p}_{seed0+p}_{ilayer}_{iphi}",phi_bins[2*iphi],phi_bins[2*iphi],hist_pt_bin.FindBin(pt_boarders[p]),hist_pt_bin.FindBin(pt_boarders[p+1])-1)
        var_porj.SetLineColor(1)

        if var_porj.GetEntries() < Min_Entries:
            continue

        pt_loc = round((pt_boarders[p]*2+pt_boarders[p+1])/3,2)
        pt_loc_err = (pt_boarders[p+1]-pt_boarders[p])/4.
        var_porj.SetTitle(f"dphi_{ilayer}_{iphi}={round(central_bean_prev,3)}_{pt_loc}")

        par_tot = array( 'd', (3*N_gaus+4)*[2.] )
        g_gaus= root.TF1( f'g_gaus_{seed0+p}_{ilayer}_{iphi}', 'gaus',  boards_fit[int(ilayer%2)][0]+central_bean_prev, boards_fit[int(ilayer%2)][1]+central_bean_prev)
        g_gaus2= root.TF1( f'g_gaus_2_{seed0+p}_{ilayer}_{iphi}', '[0]*TMath::Landau([3]*x,[1],[2])',  boards_fit2[iilayer][0]+central_bean_prev, boards_fit2[iilayer][1]+central_bean_prev)#[0]/[2]/sqrt(2*3.14159)*exp(-0.5*pow((x-[1])/[2],2))
        g_pol = root.TF1( f'g_pol{seed0+p}_{ilayer}_{iphi}', '[0]+[1]*x*(abs(x)/x-1)+[2]*x*(abs(x)/x+1)',  boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        Format_Func(g_pol,3,2)

        total = root.TF1( f'total_pos{seed0+p}_{ilayer}_{iphi}', 'gaus(0)+[3]*TMath::Landau([9]*x,[4],[5])+[6]+[7]*x*(abs(x)/x-1)+[8]*x*(abs(x)/x+1)',boards_tot[int(ilayer%2)][0], boards_tot[int(ilayer%2)][1])
        Format_Func(total,2,6)
        g_gaus2.FixParameter(3,charge)

        if fits_mean == 0:
            g_gaus.SetParameter(1,0.)
            g_gaus.SetParameter(2,boards_fit[1][1])
            g_gaus.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
            g_gaus.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
            g_gaus2.SetParameter(1,0.5*(boards_fit2[int(iilayer/2)][0]+boards_fit2[int(iilayer/2)][1])+central_bean_prev)
            g_gaus2.SetParameter(2,0.5*(boards_fit2[int(iilayer/2)][1]-boards_fit2[int(iilayer/2)][0]))
            g_gaus2.SetParLimits(0,1,10000000)
            g_gaus2.SetParLimits(1,boards_fit2[int(iilayer/2)][0]+central_bean_prev, boards_fit2[int(iilayer/2)][1]+central_bean_prev)
        else: 
            mean0 = fits_mean[0][ilayer][iphi].Eval(pt_loc)
            sigm0 = fits_sigma[0][ilayer][iphi].Eval(pt_loc)
            mean1 = mean0+charge*(0.005+0.01*math.exp(-3.33*pt_loc))*factor1
            sigm1 = sigm0*factor2
            g_gaus.SetParameter(1,mean0)
            g_gaus.SetParameter(2,sigm0)
            g_gaus2.SetParameter(1,mean1)
            g_gaus2.SetParameter(2,sigm1)

        var_porj.Draw()
        var_porj.Fit( g_gaus, 'QR')
        par_gaus = g_gaus.GetParameters()
        g_gaus2.SetParLimits(1, par_gaus[1]*charge+leftright[iilayer%2][0]*par_gaus[2], par_gaus[1]*charge+leftright[iilayer%2][1]*par_gaus[2] )
        g_gaus2.SetParLimits(2,leftright_sigma[iilayer%2][0]*par_gaus[2], leftright_sigma[iilayer%2][1]*par_gaus[2] )

        var_porj.Fit( g_gaus2, 'QR+' )
        var_porj.Fit( g_pol, 'QR+' )

        par_gaus2 = g_gaus2.GetParameters()
        par_pol = g_pol.GetParameters()

        par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5] = par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2]
        par_tot[6], par_tot[7], par_tot[8], par_tot[9] = par_pol[0], par_pol[1], par_pol[1], charge
        total.FixParameter(9,charge)

        total.SetParameters( par_tot )
        total.SetParLimits(1,boards_tot[int(ilayer%2)][0]/10+central_bean_prev,boards_tot[int(ilayer%2)][1]/10+central_bean_prev)
        total.SetParLimits(2,boards_fit[int(ilayer%2)][1]/10,2)
        total.SetParLimits(3,par_gaus2[0]*0.49, par_gaus2[0]*2)
        total.SetParLimits(4,par_gaus2[1]-0.01, par_gaus2[1]+0.01)
        total.SetParLimits(5,par_gaus2[2]*0.49, par_gaus2[2]*8)
        #if iOption0==1:
        #    total.SetParameter(3,0.)
        #    total.SetParLimits(3,0,par_gaus2[0]*0.01)
            
        if not fits_mean==0: 
            mean0 = fits_mean[0][ilayer][iphi].Eval(pt_loc)
            sigm0 = fits_sigma[0][ilayer][iphi].Eval(pt_loc)
            mean1 = mean0+charge*(0.005+0.01*math.exp(-3.33*pt_loc))*factor1
            sigm1 = sigm0*factor2
            total.SetParameter(1,mean0)
            total.SetParameter(2,sigm0)
            total.SetParameter(1,mean1)
            total.SetParameter(2,sigm1)
        #total.SetParLimits(3,1,1000000)
        if iphi>-1:
            total.SetParLimits(1,boards_tot[int(ilayer%2)][0],boards_tot[int(ilayer%2)][1])
        #if iOption0%2==1: total.FixParameter(3,0.)
        
        var_porj.Draw()
        var_porj.Fit( total, 'QR' )

        par_tot = total.GetParameters()
        mean1 = par_tot[1]+charge*(0.005+0.01*math.exp(-3.33*pt_loc))*factor1
        sigm1 = par_tot[2]*factor2
        total.SetParameter(4, mean1)
        total.SetParLimits(4, par_tot[1]*charge+leftright[iilayer%2][0]*par_tot[2]/2, par_tot[1]*charge+leftright[iilayer%2][1]*par_tot[2])
        total.SetParLimits(5, par_tot[2]*0.5, 100*par_tot[2])
        
        var_porj.Draw()
        var_porj.Fit( total, 'QR' )
        par_tot = total.GetParameters()

        g_gaus= root.TF1( f'g_gaus_4_{seed0+p}_{ilayer}_{iphi}', 'gaus',  par_tot[1]-2*par_tot[2], par_tot[1]+2*par_tot[2])
        g_gaus2= root.TF1( f'g_gaus_5_{seed0+p}_{ilayer}_{iphi}', '[0]*TMath::Landau([3]*x,[1],[2])',  charge*par_tot[4]-10*par_tot[5], charge*par_tot[4]+10*par_tot[5])
        g_gaus2.FixParameter(3,charge)

        Format_Func(g_gaus,3,4)
        Format_Func(g_gaus2,3,root.kGreen+2)

        par_gaus[0], par_gaus[1], par_gaus[2], par_gaus2[0], par_gaus2[1], par_gaus2[2] = par_tot[0], par_tot[1], par_tot[2], par_tot[3], par_tot[4], par_tot[5]
        par_pol[0], par_pol[1], par_pol[2] = par_tot[6], par_tot[7], par_tot[8]
        g_gaus2.SetParameters(par_gaus2)
        g_gaus.SetParameters(par_gaus)
        g_pol.SetParameters(par_pol)

        g_gaus.Draw('same')
        g_gaus2.Draw('same')
        g_pol.Draw('same')

        #if is_sigmalised:
        #    if abs(total.GetParameter(1))>1.5:
        #        continue
        if not is_sigmalised and iOption0==0 and pt[p]>1 and total.GetParameter(2)>0.003: continue

        
        pt_local.append(pt_loc)
        pt_local_err.append(pt_loc_err)

        Mean_local.append(total.GetParameter(1)-0*total.GetParameter(4)*charge)
        Sigma_local.append(total.GetParameter(2+3*0))
        if is_sigmalised==0:
            Mean_local_err.append (min(total.GetParError(1)+0.05*total.GetParameter(2),Sigma_local[-1]*0.5))
            Sigma_local_err.append(min(total.GetParError(2)+0.05*total.GetParameter(2),Sigma_local[-1]*0.3))
        else:
            Mean_local_err.append(min(total.GetParError(1)+ 0.1*1., 0.2))#(0.15-0.005*p)*total.GetParameter(2)
            Sigma_local_err.append(min(total.GetParError(2)+0.1*1., 0.1))#(0.15-0.005*p)*total.GetParameter(2)


        int_fg_local.append(g_gaus.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
        int_bg_local.append(g_pol.Integral(total.GetParameter(1)-sigma*total.GetParameter(2),total.GetParameter(1)+sigma*total.GetParameter(2)))
            
        #m2_pos.Draw("same") m2_neg.Draw("same")

        c1.SaveAs( "output/dynamic_delta_fits/" + m2_name + ".png" )

    return [Mean_local, Mean_local_err, Sigma_local, Sigma_local_err, int_fg_local, int_bg_local, pt_local, pt_local_err]


In [10]:
def sdphi_fitter_centr(ilayer=1, start_file=-1, last_file=0, fits_mean=0, fits_sigma=0,InHist=hists_read, phibins=[0,1,2,3,4,5], sigma=2):
    out_hists = []
    for iFile in range(start_file,last_file):
        out_hists1 = []
        for iphi in phibins:
            out_hists1.append(sdphi_fitter(InHist[iFile][ilayer][iOption0+is_sigmalised*2], iphi, iFile, ilayer, sigma, fits_mean, fits_sigma))
        out_hists.append(out_hists1)
    return out_hists

In [11]:
def sdphi_fitter_run(ilayer=1):
    return sdphi_fitter_centr(ilayer)

In [12]:
Ntr = N_layers
pool = mp.Pool(Ntr)
with pool:
    output_array = pool.map(sdphi_fitter_centr, [i for i in range(Ntr)])
pool.close()

In [13]:
Means, Mean_errs, Sigmas, Sigma_errs, Int_FG, Int_BG, pt_arr, pt_arr_err = [], [], [], [], [], [], [], []
for ifile in range(1):
    Means1, Mean_errs1, Sigmas1, Sigma_errs1, Int_FG1, Int_BG1, pt_arr1, pt_arr_err1 = [], [], [], [], [], [], [], []
    for ilayer in range(N_layers):
        Means0, Mean_errs0, Sigmas0, Sigma_errs0, Int_FG0, Int_BG0, pt_arr0, pt_arr_err0 = [], [], [], [], [], [], [], []
        for iphi in range(N_phi):
            Means0.append(output_array[ilayer][ifile][iphi][0])
            Mean_errs0.append(output_array[ilayer][ifile][iphi][1])
            Sigmas0.append(output_array[ilayer][ifile][iphi][2])
            Sigma_errs0.append(output_array[ilayer][ifile][iphi][3])
            Int_FG0.append(output_array[ilayer][ifile][iphi][4])
            Int_BG0.append(output_array[ilayer][ifile][iphi][5])
            pt_arr0.append(output_array[ilayer][ifile][iphi][6])
            pt_arr_err0.append(output_array[ilayer][ifile][iphi][7])
        Means1.append(Means0)
        Mean_errs1.append(Mean_errs0)
        Sigmas1.append(Sigmas0)
        Sigma_errs1.append(Sigma_errs0)
        Int_FG1.append(Int_FG0)
        Int_BG1.append(Int_BG0)
        pt_arr1.append(pt_arr0)
        pt_arr_err1.append(pt_arr_err0)
    Means.append(Means1)
    Mean_errs.append(Mean_errs1)
    Sigmas.append(Sigmas1)
    Sigma_errs.append(Sigma_errs1)
    Int_FG.append(Int_FG1)
    Int_BG.append(Int_BG1)
    pt_arr.append(pt_arr1)
    pt_arr_err.append(pt_arr_err1)

In [14]:
types = ["emc_r_dphi_","emc_min_r_dphi_","emc_cnt_dphi_","emc_r_dz_","emc_min_r_dz_","emc_cnt_dz_"]
fit_functions_0iter = []
for ifile in range(N_files):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_sigma_0iter_{ifile}_{iphi}_{ilayer}_{iOption0}","[0]+[1]*exp([2]*x)",0.5,5.0))
            fit_functions1[-1].SetParameter(0,0.0015)
            fit_functions1[-1].SetParameter(1,0.02)
            fit_functions1[-1].SetParameter(2,-3.3)
            fit_functions1[-1].SetParLimits(2,-8.126,-0.023)
            fit_functions1[-1].SetParLimits(0,0.001,0.002)
            if ilayer<2 : fit_functions1[-1].SetParLimits(2,-4.5,-2.5)
            elif ilayer<4 : fit_functions1[-1].SetParLimits(2,-4.0,-2.5)
            else : 
                fit_functions1[-1].SetParLimits(2,-6.0,-2.0)
                fit_functions1[-1].SetParLimits(1,0.0001,0.01)
            if iOption0%2==1: 
                fit_functions1[-1].SetParLimits(2,-5.126,-1.23)
                fit_functions1[-1].SetParameter(1,0.02)
                fit_functions1[-1].SetParLimits(1,0.001,0.6)
                fit_functions1[-1].SetParameter(0,0.005)
                fit_functions1[-1].SetParLimits(0,0.001,0.006)
            if is_sigmalised: 
                fit_functions1[-1].SetParLimits(0,0.,20)
                fit_functions1[-1].SetParLimits(1,0.,20)
                fit_functions1[-1].SetParLimits(2,-30,+10.23)
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_functions_0iter.append(fit_functions0)

In [15]:
g_sigma = []
if not is_sigmalised: N_layers = 16
for ifile in range(1):
    g_sigma_f = []
    h2=[]
    c3 = root.TCanvas(f"c3{iFile}",f"c3{iFile}",1400,125*N_layers)
    c3.Divide(2,int(N_layers/2),0,0)
    Min, Max = 0., 0.01
    if is_sigmalised==1: Max, Min = 1.99, 0.01
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        g_sigma_layer = []
        h2.append(Format_Pad_old( 0.05, 5, Min, Max, "p_{T} (GeV/#it{c})", "#sigma#"+iOptames[iOption0], 1.1, 1.1, 0.06, 0.06, "",0.15,0.15,0.,0.1))
        root.gPad.SetTopMargin(0.0)
        root.gPad.SetRightMargin(0.0)
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.5,0.95,0.98,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_sigma_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Sigmas[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Sigma_errs[ifile][ilayer][iphi]))
            g_sigma_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_sigma_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if iOption0 == 0 and ilayer%2==0 and (iphi == 0 or iphi == N_phi-1): 
                g_sigma_layer[-1].Fit(fit_functions_0iter[ifile][ilayer][iphi],"Q","",0.9,1.2)
            else: g_sigma_layer[-1].Fit(fit_functions_0iter[ifile][ilayer][iphi],"Q","",0.3,5)
            g_sigma_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_sigma_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_sigma_f.append(g_sigma_layer)
    g_sigma.append(g_sigma_f)
    c3.Draw()
    #c3.SaveAs(f"output/note/sigma_{ilayer}{iOption0}{is_sigmalised}.pdf")

In [16]:
fit_mean_functions_0iter = []
for ifile in range(1):
    fit_functions0 = []
    for ilayer in range(N_layers):
        fit_functions1 = []
        for iphi in range(N_phi):
            fit_functions1.append(root.TF1(f"fit_functions_mean_0iter_{ifile}_{iphi}_"+layers[ilayer],"[0]*(1+[1]*exp([2]*x))",0.0,5.0))
            if iOption0<2:
                fit_functions1[-1].SetParameter(1,1)
                if ilayer < 4: 
                    fit_functions1[-1].FixParameter(2,-1)
                else:
                    fit_functions1[-1].FixParameter(1,-1)
                fit_functions1[-1].SetParameter(1,1)
                        
            fit_functions1[-1].SetLineColor(colors[iphi])
        fit_functions0.append(fit_functions1)
    fit_mean_functions_0iter.append(fit_functions0)

In [17]:
g_mean = []
for iFile in range(1):
    g_mean_f = []
    h2=[]
    c3 = root.TCanvas(f"c2{iFile}",f"c2{iFile}",1400,125*N_layers)
    c3.Divide(2,int(N_layers/2),0,0)
    Min, Max = -0.05, 0.05
    if is_sigmalised==1: Max, Min = 1.5, -1.5
    names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
    legends =[]
    for ilayer in range(N_layers):
        c3.cd(ilayer+1)
        g_mean_layer = []
        h2.append(Format_Pad_old( 0.05, 5, Min, Max, "p_{T} (GeV/#it{c})", "#LT#"+iOptames[iOption0]+"#GT", 1.1, 1.1, 0.06, 0.05, "",0.15,0.15,0.,0.1))
        root.gPad.SetTopMargin(0.0)
        root.gPad.SetRightMargin(0.0)
        h2[-1].Draw()
        legends.append(root.TLegend(0.55,0.4,0.85,0.9,layers[ilayer]))
        legends[-1].SetFillColorAlpha(0,0)
        legends[-1].SetLineWidth(0)
        for iphi in range(0,N_phi):
            N_pt=len(pt_arr[ifile][ilayer][iphi])
            if N_pt<1: continue
            g_mean_layer.append(root.TGraphErrors(N_pt, pt_arr[ifile][ilayer][iphi], Means[ifile][ilayer][iphi],pt_arr_err[ifile][ilayer][iphi],Mean_errs[ifile][ilayer][iphi]))
            g_mean_layer[-1].SetName(f"sigma_o{iOption0}_s{sim}_f{ifile}_c{ilayer}_l{iphi}")
            Format_Graph(g_mean_layer[-1],20+int(iphi/4),1, colors[iphi], 3, colors[iphi],1,1)
            if not is_sigmalised and iOption0==0: g_mean_layer[-1].Fit(fit_mean_functions_0iter[ifile][ilayer][iphi],"Q","",0.9,1.2)
            else: g_mean_layer[-1].Fit(fit_mean_functions_0iter[ifile][ilayer][iphi],"Q","",0.4,3)
            g_mean_layer[-1].Draw('P')
            central_bean_prev = round(0.002*phi_bins[2*iphi]-0.001-0.1,3)
            legends[-1].AddEntry(g_mean_layer[-1],f"phi_prev {central_bean_prev}","p")
        legends[-1].Draw()
        g_mean_f.append(g_mean_layer)
    g_mean.append(g_mean_f)
    c3.Draw()
    #c3.SaveAs(f"output/note/mean_{ilayer}{iOption0}{is_sigmalised}.pdf")
    #c3.SaveAs(f"output/Sigmas/mean_{ilayer}{iOption0}{is_sigmalised}.png")

In [18]:
dynamic_dphi = []
dynamic_dphi_err = []
dphi = []
dphi_err = []

for ifile in range(0,1):
    dynamic_dphi_ifile = []
    dynamic_dphi_err_ifile = []
    dphi_ifile = []
    dphi_err_ifile = []
    for ilayer in range(N_layers):
        dynamic_dphi_ilayer = array( 'd' )
        dynamic_dphi_err_ilayer = array( 'd' )
        dphi_ilayer = array( 'd' )
        dphi_err_ilayer = array( 'd' )
        for iphi in range(N_phi):
            if iOption0==0 and not is_sigmalised:
                if abs( fit_mean_functions_0iter[ifile][ilayer][iphi].GetParameter(0) - ((0.002*phi_bins[2*iphi]-0.001-0.1))) > 0.0025: continue
                if ilayer>7 and (ilayer%4<2 and iphi<1) or (ilayer%4>1 and iphi>3) : continue
            elif iOption0==1 and not is_sigmalised:
                if (iphi<1 or iphi>4): continue
                if abs( fit_mean_functions_0iter[ifile][ilayer][iphi].GetParameter(0) - ((0.002*phi_bins[2*iphi]-0.001-0.1))) > 0.005: continue
            if not is_sigmalised and not new: dphi_ilayer.append( round(0.001*phi_bins[2*iphi]-0.0005-0.1,5))
            if new and not is_sigmalised: dphi_ilayer.append( round(0.002*phi_bins[2*iphi]-0.001-0.1,5))
            if is_sigmalised: 
                dphi_ilayer.append(round(0.2*phi_bins[2*iphi]-0.1-10,5))
            dphi_err_ilayer.append(0.001/4)
            dynamic_dphi_ilayer.append(fit_mean_functions_0iter[ifile][ilayer][iphi].GetParameter(0))
            dynamic_dphi_err_ilayer.append(max(0.0001,fit_mean_functions_0iter[ifile][ilayer][iphi].GetParError(0)))
        dynamic_dphi_ifile.append(dynamic_dphi_ilayer)
        dynamic_dphi_err_ifile.append(dynamic_dphi_err_ilayer)
        dphi_ifile.append(dphi_ilayer)
        dphi_err_ifile.append(dphi_err_ilayer)
    dynamic_dphi.append(dynamic_dphi_ifile)
    dynamic_dphi_err.append(dynamic_dphi_err_ifile)
    dphi.append(dphi_ifile)
    dphi_err.append(dphi_err_ifile)

print("dphi = ", dphi)
print(dynamic_dphi)

dphi =  [[array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1]), array('d', [-1.1, -0.9, -0.5, 0.1, 0.9, 1.1])]]
[[array('d', [-0.000537345699593776, -0.0001849094652134405, -3.077860978314409e-05, 0.17201224752286143, 0.15317480152498156, 0.1509798742526938]), array('d', [-1.0978267178500531e-05, -1.1736446361256209e-05, -7.149264237787195e-06, 0.

In [19]:
g_mean_phi = []
c4 = root.TCanvas(f"c40",f"c40",500,450)
h3=[]
legends =[]
for ifile in range(1):
    c4.cd(1+ifile)
    g_mean_phi_type=[]
    for itype in range(1):
        Min, Max, x0, x1 = -0.05, 0.05, -0.02, 0.05
        if is_sigmalised==1: Max, Min, x0, x1 = 1+itype*2, -1-itype*2, -2, 5
        names = ["emc_r_dphi","emc_min_r_dphi","emc_cnt_dphi","emc_r_dz","emc_min_r_dz","emc_cnt_dz"]
        for iphi in range(1):
            h3.append(Format_Pad_old( x0, x1, Min, Max, f"{iOptames[iOption0]}_prev [rad]", "#LT"+iOptames[iOption0]+"#GT [rad]", 1.1, 1.1, 0.06, 0.05, "",0.2,0.2,0.,0.1))
            h3[-1].Draw()
            legends.append(root.TLegend(0.55,0.4,0.85,0.9))
            legends[-1].SetFillColorAlpha(0,0)
            legends[-1].SetLineWidth(0)
            g_mean_phi_layer=[]
            for ilayer in range(0,N_layers):
                N_phi = len(dphi[ifile][ilayer])
                g_mean_phi_layer.append(root.TGraphErrors(N_phi, dphi[ifile][ilayer], dynamic_dphi[ifile][ilayer],dphi_err[ifile][ilayer],dynamic_dphi_err[ifile][ilayer]))
                g_mean_phi_layer[-1].SetName(f"test_graph_{itype}_{ifile}_{ilayer}_{iphi}_{iOption0}")

                Format_Graph(g_mean_phi_layer[-1],20+int(ilayer/4),1, colors[ilayer%4], 3, colors[ilayer%4],1,1)
                g_mean_phi_layer[-1].Fit("pol1","Q","")
                g_mean_phi_layer[-1].GetFunction("pol1").SetLineColor(colors[ilayer%4])
                g_mean_phi_layer[-1].Draw('sameP')
                legends[-1].AddEntry(g_mean_phi_layer[-1],layers[ilayer],"p")

            legends[-1].Draw()
        g_mean_phi_type.append(g_mean_phi_layer)
    g_mean_phi.append(g_mean_phi_type)
c4.Draw()
c4.SaveAs(f"output/Sigmas/mean_phi_prevphi_{ifile}{iOption0}{is_sigmalised}.png")


In [20]:
if True:
    print("const float "+iOptames[iOption0]+f"_sigma_pt_params[{N_layers}][3]="+"\n{")
    for ilayer in range(len(fit_functions_0iter[0])):
        fits = fit_functions_0iter[0][ilayer]
        if iOption0%2==0:
            if ilayer < 8 :
                print("    {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
            elif ilayer%4<2:
                print("    {"+f"{round((fits[4].GetParameter(0)+fits[4].GetParameter(0))*0.5,6)}, {round((fits[4].GetParameter(1)+fits[4].GetParameter(1))*0.5,6)}, {round((fits[4].GetParameter(2)+fits[4].GetParameter(2))*0.5,6)}"+"},")
            else:
                print("    {"+f"{round((fits[0].GetParameter(0)+fits[1].GetParameter(0))*0.5,6)}, {round((fits[0].GetParameter(1)+fits[1].GetParameter(1))*0.5,6)}, {round((fits[0].GetParameter(2)+fits[1].GetParameter(2))*0.5,6)}"+"},")
        else:
            print("    {"+f"{round((fits[2].GetParameter(0)+fits[3].GetParameter(0))*0.5,6)}, {round((fits[2].GetParameter(1)+fits[3].GetParameter(1))*0.5,6)}, {round((fits[2].GetParameter(2)+fits[3].GetParameter(2))*0.5,6)}"+"},")
    print("};")
print("const float "+iOptames[iOption0]+f"_mean_"+iOptames[iOption0]+f"_params[{N_layers}][2]="+"\n{")
for ilayer in range(N_layers):
    print("    {"+f'{round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(0),6)}, {round(g_mean_phi[ifile][0][ilayer].GetFunction("pol1").GetParameter(1),6)}'+"},")
print("};")


const float phi_sigma_pt_params[16][3]=
{
    {0.344004, 0.479701, -0.174287},
    {0.420532, 0.412167, -2.204082},
    {0.607931, 0.273471, -0.253839},
    {0.599473, 0.278103, -0.224692},
    {0.000147, 0.942308, -0.12578},
    {0.374349, 0.698495, -0.495137},
    {0.360772, 0.678954, -0.551934},
    {0.677989, 0.39692, -0.567012},
    {0.0, 1.092676, -0.236081},
    {0.0, 1.082723, -0.172779},
    {0.570167, 1.10943, -1.415593},
    {0.241016, 0.78089, -0.177722},
    {0.0, 0.957854, -0.307123},
    {0.306193, 0.922569, -0.786104},
    {0.559507, 0.915573, -1.774433},
    {0.226402, 0.855596, -0.469377},
};
const float phi_mean_phi_params[16][2]=
{
    {0.000405, 0.000846},
    {-0.0, 1.1e-05},
    {4.2e-05, 8.6e-05},
    {1.5e-05, 2.7e-05},
    {-0.005554, 0.017744},
    {-22588197.210843, 174828186905.36044},
    {11654954865.51698, 174828268276.2377},
    {11654971478.021128, 174828267848.00427},
    {11655217850.789701, 174828267848.00427},
    {11655217850.789701, 174828267848.